# *Biomphalaria* hemolymph microbiome


## Aim

This notebook contains the steps to install the environment, download the data and perform the data analysis related to the study of the hemololymph microbiome of *Biomphalaria* snails. Results are presented in the manuscript entitled "The hemolymph of *Biomphalaria* snail vectors of schistosomiasis supports a diverse microbiome". 

Several folders are already available or will be created during the analysis. Here is the list of the folders and their content:
* **data**: Sequencing files, qPCR result file, database for taxonomy assignment, sample metadata, list of contaminants and list of the main bacteria found in whole snail microbiomes. These files will be either present or downloaded.
* **.env**: Files needed to create the environment.
* **graphs**: Graphical representation of the data. If not existing, this will be created during the analysis.
* **results**: Files that are generated through data processing. If not existing, the folder will be created during the analysis.
* **scripts**: Scripts used for the analysis.


## Environment and data

### Creating environment

In [ ]:
# Check if conda is available
[[ ! $(which conda 2> /dev/null) ]] && echo "conda not available in \$PATH. Please interrupt the kernel and fix the situation." && sleep inf

# Creating conda environment
conda env create -f .env/env.yml

The cell below must be run each time a new Jupyter session is run.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate ubiome_hml

The cell below must be run only once at the time of the environment creation.

In [ ]:
# Installing needed R packages
Rscript ".env/R package dependencies.R"

### Downloading sequencing data

This step downloads the fastq files of the different samples from the SRA repository.

In [ ]:
# Data directory
ldir="data/libraries"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Bioproject
bioproject=PRJNA613098

# Download related information to data project
wget -q -O runinfo "http://trace.ncbi.nlm.nih.gov/Traces/sra/sra.cgi?save=efetch&rettype=runinfo&db=sra&term=${bioproject}"

# Field of interest (library name and weblink)
fdn=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "LibraryName" | cut -d ":" -f 1)
fdr=$(head -n 1 runinfo | tr "," "\n" | grep -w -n "Run" | cut -d ":" -f 1)

# Download fastq files
while read line
do
    # Filename and download link
    fln=$(cut -d "," -f $fdn <<<$line)
    run=$(cut -d "," -f $fdr <<<$line)
    
    # Download
    echo "$fln"
    fastq-dump -O "$ldir" --split-files "$run"
    
    mv "$ldir/${run}_1.fastq" "$ldir/${fln}_R1.fastq"
    mv "$ldir/${run}_2.fastq" "$ldir/${fln}_R2.fastq"
        
done < <(tail -n +2 runinfo | head -1)

# Compress files
pigz "$ldir/"*

rm runinfo

### Downloading database

The Silva database will be used to assign taxonomy to the amplicon sequence variants (ASVs) generated from the sequencing data.

In [ ]:
# Database directory
dbdir="data/Silva db"
[[ ! -d "$dbdir" ]] && mkdir -p "$dbdir"

# Download and extract the relevant Silva file
wget -P "$dbdir" 'https://www.arb-silva.de/fileadmin/silva_databases/qiime/Silva_132_release.zip'
unzip "$dbdir/Silva_132_release.zip" -d "$dbdir" && rm "$dbdir/Silva_132_release.zip"

# Import the sequence database in Qiime format
qiime tools import \
    --input-path "$dbdir/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna" \
    --output-path "$dbdir/silva_132_99_16S.qza" \
    --type 'FeatureData[Sequence]'

# Import the taxonomy database in Qiime format
qiime tools import \
    --input-path "$dbdir/SILVA_132_QIIME_release/taxonomy/16S_only/99/taxonomy_all_levels.txt" \
    --output-path "$dbdir/silva_132_99_16S_taxa.qza" \
    --type 'FeatureData[Taxonomy]' \
    --input-format HeaderlessTSVTaxonomyFormat

## Qiime pipeline

This section processes the data to generate ASVs and assign taxonomy.

In [ ]:
# Qiime output directory
qdir="results/1-qiime"
[[ ! -d "$qdir" ]] && mkdir -p "$qdir"

# Metadata file
metadata="data/sample-metadata.tsv"

### Sequencing data

If sequencing data are present, they will be imported as a Qiime artifact.

In [ ]:
# Check for sequencing data
[[ ! $(find "$ldir" -type f -name *fastq.gz) ]] && echo  "No sequencing data. Please interrupt the kernel and fix the situation." && sleep inf

# Create the manifest for importing data in artefact
## source: https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats
for i in $(ls "$ldir"/* | sed "s,_R[12].fastq.*,,g" | uniq)
do
    nm=$(sed "s,$ldir/,," <<<$i)
    fl=$(ls -1 "$PWD/$i"* | tr "\n" "\t")

    echo -e "$nm\t$fl"
done > "$qdir/manifest"

# Add header
sed -i "1s/^/sample-id\tforward-absolute-filepath\treverse-absolute-filepath\n/" "$qdir/manifest"

# Import data
## source: https://docs.qiime2.org/2019.4/tutorials/importing/
qiime tools import \
    --type 'SampleData[PairedEndSequencesWithQuality]' \
    --input-path "$qdir/manifest" \
    --input-format PairedEndFastqManifestPhred33V2 \
    --output-path "$qdir/demux-paired-end.qza"

**Note**:
* No need to remove adapters and barcodes. This has been done during `bcl2fastq`. One can check it using `grep`.
* Importing with `--input-format PairedEndFastqManifestPhred33V2` instead of `--input-format CasavaOneEightSingleLanePerSampleDirFmt` for [custom sample names](https://docs.qiime2.org/2019.4/tutorials/importing/#fastq-manifest-formats).

### Data quality

This step generates a visualization file for checking the data quality. The visualization can be displayed on [Qiime2 website](https://view.qiime2.org/)

In [ ]:
# Make a summary to check read quality
qiime demux summarize \
    --i-data "$qdir/demux-paired-end.qza" \
    --o-visualization "$qdir/demux-paired-end.qzv"

**Note**: read quality drops toward the end but is still above 10. So no trimming done.

### Sequence clustering and denoising

This step generates ASVs from the sequencing data using the `dada2` plugin. Reads were not truncated and a maximum expected error of 5 was selected.

In [ ]:
[[ -f "$qdir/table.qza" ]] && echo "ASV file (table.qza) exists already. Please interrupt the kernel and fix the situation." && sleep inf

# Run dada2
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs "$qdir/demux-paired-end.qza" \
    --p-trunc-len-f 250 \
    --p-trunc-len-r 250 \
    --p-max-ee 5 \
    --p-n-threads 0 \
    --o-table "$qdir/table.qza" \
    --o-representative-sequences "$qdir/rep-seqs.qza" \
    --o-denoising-stats "$qdir/denoising-stats.qza"

# Add metadata information to the denoising stats
qiime metadata tabulate \
    --m-input-file "$metadata" \
    --m-input-file "$qdir/denoising-stats.qza" \
    --o-visualization "$qdir/denoising-stats.qzv"

### Taxonomy identification

This step assigns taxonomy to the ASVs generated. This is done using the Silva database and the `classify-consensus-vsearch` method from the `feature-classifier` plugin with an identity threshold of 97%.

In [ ]:
qiime feature-classifier classify-consensus-vsearch \
    --i-query "$qdir/rep-seqs.qza" \
    --i-reference-reads "$dbdir/silva_132_99_16S.qza" \
    --i-reference-taxonomy "$dbdir/silva_132_99_16S_taxa.qza" \
    --p-perc-identity 0.97 \
    --p-threads $(nproc) \
    --o-classification "$qdir/rep-seqs_taxa.qza"

### Phylogeny

This step generates a phylogeny from the ASVs ([source](https://chmi-sops.github.io/mydoc_qiime2.html)). ASVs are aligned, masked for highly variable regions. A tree is created from the alignment and rooted in its center.

In [ ]:
[[ -f "$qdir/rooted-tree.qza" ]] && echo "A tree file (rooted-tree.qza) exists already. Please interrupt the kernel and fix the situation." && sleep inf

# Multiple seqeunce alignment using Mafft
qiime alignment mafft \
    --i-sequences "$qdir/rep-seqs.qza" \
    --o-alignment "$qdir/aligned-rep-seqs.qza"

# Masking (or filtering) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree.
qiime alignment mask \
    --i-alignment "$qdir/aligned-rep-seqs.qza" \
    --o-masked-alignment "$qdir/masked-aligned-rep-seqs.qza"

# Creating tree using the Fasttree program
qiime phylogeny fasttree \
    --i-alignment "$qdir/masked-aligned-rep-seqs.qza" \
    --o-tree "$qdir/unrooted-tree.qza"

# Root the tree using the longest root
qiime phylogeny midpoint-root \
    --i-tree "$qdir/unrooted-tree.qza" \
    --o-rooted-tree "$qdir/rooted-tree.qza"

## Library analysis

To better understand the impact of the different steps on the number of reads filtered out, we analyze the Qiime2 visualization generated after the denoising step. We generate a table summarizing number of reads retained (mean, standard deviation and range) at each step, which includes the number of initial, filtered, denoised, merged and non chimeric reads, for each population and replicate.

In [ ]:
scripts/library_stats.R

## Identification of ASVs with unassigned taxonomy

ASVs with unassigned taxonomy could correspond to eukaryotic contaminants because of the 16S primers amplifying on the 5S or 18S regions. To exclude such contaminants, we perform a megablast search againt the NCBI nt database to find the best sequence similarity to a given unassigned ASV. The results will be then used to exclude non 16S sequences for the subsequent analysis.

Because of the time this analysis can take (up to 7h), the list of contaminants is already available in the data folder.

### Export of unassigned ASV

We use Qiime to filter and export ASV without taxonomy assignments.

In [ ]:
# Filtering assigned ASV out 
qiime taxa filter-seqs \
    --i-sequences "$qdir/rep-seqs.qza" \
    --i-taxonomy "$qdir/rep-seqs_taxa.qza" \
    --p-include "Unassigned" \
    --o-filtered-sequences "$qdir/rep-seqs_unassigned.qza"

# Exporting sequences in fasta format
qiime tools export \
    --input-path "$qdir/rep-seqs_unassigned.qza" \
    --output-path "$qdir/rep-seqs_unassigned"

### Blast and annotation of unassigned ASVs

To identify the most similar sequences to the unassigned ASVs, we perform a megablast. This is done using a relatively lenient e-value parameter (1e-2) to increase power of detection. This step is relatively long (estimated running time: 5 h - 8 h) even when using a maxiumum target of 1 with a maximum alignment (HSPs) of 1. The resulting table is then updated with the title and the phyla of the blast match (estimated running time: 30 min - 1 h). Finally we correct the annotations because some ASVs match 16S mitochondrial DNA of eukayote organisms and are wrongly classified as eukaryotes while they likely represent bacteria.

In [ ]:
# Blast the unassigned against the nt database
blastn -task megablast -db nt -remote \
    -query "$qdir/rep-seqs_unassigned/dna-sequences.fasta" \
    -max_target_seqs 1 \
    -max_hsps 1 \
    -evalue 1e-2 \
    -outfmt 6 > "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"

# Complete the table to identify what kind of organism the match belongs to
for ((i=1; i <= $(wc -l < "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"); i++))
do
    # Get GI from the blast result
    gi=$(sed -n "${i}p" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv" | cut -f 2)
    
    # Download entry and get title and phylym info
    entry=$(wget -q -O - "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=${gi}&rettype=gb")
    title=$(echo "$entry" | grep "^DEFINITION" | cut -d " " -f 3-)
    phylum=$(echo "$entry" | grep -A 1 "^ *ORGANISM" | sed -n "2p" | cut -d ";" -f 1 | sed "s/ *//g")
    
    # Update table line
    sed -i "${i}s/$/\t$title\t$phylum/" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"
    
    # Sleep a little to avoid server closing connection on next request
    sleep 0.25s
done

# Correct annotation
for i in $(egrep -n "mitochondri.*Eukaryota$" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv" | cut -d ":" -f 1)
do
    sed -i "${i}s/Eukaryota/Bacteria/" "$qdir/rep-seqs_unassigned/unassigned.blastn.tsv"
done

## Microbiome diversity

We analyze the ASV diversity using different approaches (rarefaction curve, $\alpha$ and $\beta$ diversity, rare taxa). Technical replicates are also evaluated to show the robustness of the library generation method. Details about the methods used are in the R script. Analysis of the results are detailed in the manuscript.

In [ ]:
Rscript scripts/microbiome_diversity.R

### Comparison with whole snail microbiomes

To compare our results to previous microbiome studies, we investigate the presence of the main taxa (genus or family) found in these studies. This qualitative analysis aims to reveal if at least one sample in each population harbors the searched taxa. We choose the genus as the lowest rank targeted because species identification is often inaccurate. The absence of taxa may result from a true absence or from differences in the protocols (16S region amplified, primers, database, pipeline used) which may have precluded identification from our dataset.

List of taxa and related publications are present in the `whole_snails_main_bacteria.tsv` table located in the data folder.

In [ ]:
Rscript scripts/microbiome_comparison.R

## Functional inference

We investigate the potential role of the hemolymph microbiome by analyzing the metabolic pathways differentially represented between hemolymph and water samples. The pathway inference is done using PiCRUST2 and following this [tutorial](https://github.com/picrust/picrust2/wiki/PICRUSt2-Tutorial-(v2.1.4-beta)#pathway-level-inference).

In [ ]:
# PiCRUST output directory
pdir="results/2-picrust2"
[[ ! -d "$pdir" ]] && mkdir -p "$pdir"

In [ ]:
# Filter table to retain sample from each replicate
for i in {1..2}
do
    awk -v i=$i 'NR==1; $5 == i {print}' "$metadata" > "$qdir/.metadata"
    
    qiime feature-table filter-samples \
        --i-table "$qdir/table.qza"  \
        --m-metadata-file "$qdir/.metadata" \
        --o-filtered-table "$qdir/table_rep$i.qza"
done

# Clean
rm "$qdir/.metadata"

In [ ]:
# Export sequences
qiime tools export \
    --input-path "$qdir/rep-seqs.qza" \
    --output-path "$pdir/"

# Remove contaminants
while read i
do
    sed -i "/>${i}/,+1d" "$pdir/dna-sequences.fasta"
done < "data/contaminants"

for i in {1..2}
do
    qiime tools export \
        --input-path "$qdir/table_rep$i.qza" \
        --output-path "$pdir/rep$i"
done

In [ ]:
# Place reads into reference tree
place_seqs.py -s "$pdir/dna-sequences.fasta" -o "$pdir/out.tre" -p $(nproc) \
    --intermediate "$pdir/intermediate/place_seqs"

# Hidden-state prediction of gene families
hsp.py -i 16S -t "$pdir/out.tre" -o "$pdir/marker_predicted_and_nsti.tsv.gz" -p $(nproc) -n
hsp.py -i EC -t "$pdir/out.tre" -o "$pdir/EC_predicted.tsv.gz" -p $(nproc) -n

# Number of outliers
zcat "$pdir/marker_predicted_and_nsti.tsv.gz"  | tail -n +2 | awk '$3 >= 2' | wc -l

In [ ]:
# Run for each replicate
for i in {1..2}
do
    # Generate metagenome predictions
    metagenome_pipeline.py \
        -i "$pdir/rep$i/feature-table.biom" \
        -m "$pdir/marker_predicted_and_nsti.tsv.gz" \
        -f "$pdir/EC_predicted.tsv.gz" \
        --max_nsti 2.0 \
        -o "$pdir/rep$i/EC_metagenome_out" \
        --strat_out --metagenome_contrib

    # Pathway-level inference
    pathway_pipeline.py \
        -i "$pdir/rep$i/EC_metagenome_out/pred_metagenome_strat.tsv.gz" \
        -o "$pdir/rep$i/pathways_out" -p $(nproc)

    #Add functional descriptions
    add_descriptions.py -i "$pdir/rep$i/EC_metagenome_out/pred_metagenome_unstrat.tsv.gz" -m EC \
                        -o "$pdir/rep$i/EC_metagenome_out/pred_metagenome_unstrat_descrip.tsv.gz"

    add_descriptions.py -i "$pdir/rep$i/pathways_out/path_abun_unstrat.tsv.gz" -m METACYC \
                        -o "$pdir/rep$i/pathways_out/path_abun_unstrat_descrip.tsv.gz"
done

We then analyze the differences in pathway representation using the ALDEx2 R package. This analyze takes several minutes to complete.

In [ ]:
Rscript scripts/pathway_analysis.R

## Microbiome density

We investigate if the differences observed between population microbial diversity could be explained by microbe density. We analyze the results of the 16S qPCR performed on the samples used for generating the libraries and correlate the number of ASVs identified to the number of estimated bacteria.

In [ ]:
# qPCR output directory
ddir="data/qPCR"
[[ ! -d "$ddir" ]] && echo "$ddir and qPCR data are missing. Please interrupt the kernel and fix the situation." && sleep inf

# Analyze data
Rscript scripts/microbiome_density.R